In [1]:
import numpy as np
from matplotlib import pyplot as plt
from statistics import mode

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix



In [ ]:
(Xtr, Ltr), (X_test, L_test)=mnist.load_data()

In [ ]:
Xtr.shape

In [ ]:
Image=Xtr[0,:,:]
Label=Ltr[0]


plt.title('Label is {Label}'.format(Label=Label))
plt.imshow(Image)

plt.show()
plt.close()


In [ ]:
fig, ax=plt.subplots(nrows=1, ncols=2, figsize=(15,15))
ax0=plt.subplot(2,2,1)
ax1=plt.subplot(2,2,2)

Image=Xtr[0,:,:]
Label=Ltr[0]
Image1=Xtr[1,:,:]
Label1=Ltr[1]


ax0.set_title('Label is {Label}'.format(Label=Label))
ax0.imshow(Image)
ax1.set_title('Label is {Label}'.format(Label=Label1))
ax1.imshow(Image1)


plt.show()
plt.close()


In [ ]:
28*28

In [ ]:
#Traing phase
num_sample=500
Tr_set=Xtr[:num_sample,:,:]
Ltr_set=Ltr[:num_sample]

#Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2])

Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2]).astype(int)
Tr_set.shape

In [ ]:
def predict(X):
    num_test=X.shape[0]
    Lpred=np.zeros(num_test, dtype=Ltr_set.dtype)
    
    for i in range(num_test):
        #distances=np.sum(np.abs(Tr_set-X[i,:]),axis=1)
        distances=np.sqrt(np.sum((Tr_set-X[i,:])**2,axis=1))
        
        min_index= np.argmin(distances)
        Lpred[i]=Ltr_set[min_index]
    return Lpred

In [ ]:
def KNNpredict(X, K):
    num_test=X.shape[0]
    Lpred=np.zeros(num_test, dtype=Ltr_set.dtype)
    
    for i in range(num_test):
        #distances=np.sum(np.abs(Tr_set-X[i,:]),axis=1)
        distances=np.sqrt(np.sum((Tr_set-X[i,:])**2,axis=1))
        min_index= np.argsort(distances)[:K]
        Lpred[i]=mode(Ltr_set[min_index])
    return Lpred

In [ ]:
Test_images=X_test.reshape(X_test.shape[0],X_test.shape[1]* X_test.shape[2])
Labels_predicted=KNNpredict(Test_images,1)

print("Accuracy:", np.mean(Labels_predicted==L_test))



In [ ]:
from keras.datasets import cifar10

In [ ]:
(X1_train, y1_train), (X1_test, Y1_test) = cifar10.load_data()
X1_train.shape

In [ ]:
X = X1_train[:25]

In [ ]:
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]*X.shape[3])

In [ ]:
print(X.shape)

In [ ]:
def plotImage(X):
    plt.figure(figsize =(1.5, 1.5))
    plt.imshow(X.reshape(32,32,3))
    plt.show()
    plt.close()

In [ ]:
plotImage(X[24,:])

In [ ]:
from sklearn.model_selection import KFold


def predictFoldKNN(train_data, val_data, label_train, k):
    num_test= val_data.shape[0]
    Lpred=np.zeros(num_test, dtype=label_train.dtype)

    for i in range(0,num_test):
        distances=np.sqrt(np.sum(np.square(np.abs(train_data-val_data[i,:])),axis=1))

        min_index= np.argsort(distances)[:k]
        Lpred[i]=mode(label_train[min_index])

    return Lpred

def KCrossValidation(train,label,k):
    kf = KFold(n_splits=k)
    accuracy = []
    best_K = []
    #Try which K gives the best accuracy, testing K: 1 - 20
    for i in range(1,21):
        for train_index, val_index in kf.split(train):
            train_data, val_data = train[train_index], train[val_index]
            label_data, label_val_data = label[train_index], label[val_index]

            predicted = predictFoldKNN(train_data, val_data, label_data, i)
            accuracy.append(np.mean(predicted==label_val_data))
        best_K.append(np.mean(accuracy))

    return best_K
#Test_images=X_test.reshape(X_test.shape[0],X_test.shape[1]* X_test.shape[2])
#Labels_predicted_k=predict(Test_images)

#print("Accuracy:", np.mean(KCrossValidation(Tr_set, Ltr_set,3)))
a = KCrossValidation(Tr_set, Ltr_set,3)
print(a)
#Test accuracy for test images with k = 3

In [ ]:
X_norm = X/255
print("X.min()", X_norm.min())
print("X.max()", X_norm.max())

In [ ]:
X_norm.mean(axis=0)

In [ ]:
X_norm = X_norm - X_norm.mean(axis=0)

In [ ]:
X_norm.mean(axis=0)

In [ ]:
cov = np.cov(X_norm, rowvar =False)

In [ ]:
cov.shape

In [ ]:
u,s,v = np.linalg.svd(cov)

In [ ]:
print(u.shape, s.shape)

In [ ]:
print(np.diag(s))

In [ ]:
epsilon = 0.1
X_ZCA = u.dot(np.diag(1.0/np.sqrt(s+epsilon))).dot(u.T).dot(X_norm.T).T

In [ ]:
X_ZCA_rescaled = (X_ZCA - X_ZCA.min()) / (X_ZCA.max() - X_ZCA.min())
print("min", X_ZCA_rescaled.min())
print("max", X_ZCA_rescaled.max())

In [ ]:
plotImage(X[12,:])
plotImage(X_ZCA_rescaled[12,:])

In [ ]:
t = np.load("vecs.npy", allow_pickle=True)
labels = []
embeddings = []



In [ ]:
def createArrays(pos):
    for i in t.item()[pos]:
        for j in t.item()[pos][i]:
            labels.append(i)
            embeddings.append(j)

def randomPerm(x, y):
    p = np.random.permutation(len(x))
    return x[p], y[p]

In [ ]:
createArrays('1_pos')
numpyLabel = np.array(labels)
numpyEmbedding = np.array(embeddings)
numpyLabel, numpyEmbedding = randomPerm(numpyLabel, numpyEmbedding)
print(numpyLabel)
print(numpyEmbedding)

In [ ]:
iris = datasets.load_iris()
x = iris.data
y = iris.target
class_names = iris.target_names
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, train_size = 0.8)

In [ ]:
linearOVO = svm.SVC(kernel = "linear",decision_function_shape = "ovo").fit(x_train, y_train)
linearOVR = svm.SVC(kernel = "linear",decision_function_shape = "ovr").fit(x_train, y_train)
polyOVO = svm.SVC(kernel = "poly",decision_function_shape = "ovo").fit(x_train, y_train)
polyOVR = svm.SVC(kernel = "poly",decision_function_shape = "ovr").fit(x_train, y_train)
rbfOVO = svm.SVC(kernel = "rbf",decision_function_shape = "ovo").fit(x_train, y_train)
rbfOVR = svm.SVC(kernel = "rbf",decision_function_shape = "ovr").fit(x_train, y_train)


In [ ]:
linOVOPred = linearOVO.predict(x_test)
disp = confusion_matrix(linOVOPred, y_test)
print("linear One vs One")
print(disp)
print("f1 score = ", f1_score(y_test, linOVOPred, average = "weighted"), "Accuracy =", np.mean(linOVOPred == y_test))

linOVRPred = linearOVR.predict(x_test)
disp2 = confusion_matrix(linOVRPred, y_test)
print("linear One vs Rest")
print(disp2)
print("f1 score = ", f1_score(y_test, linOVRPred, average = "weighted"), "Accuracy =", np.mean(linOVRPred == y_test))

polyOVOPred = polyOVO.predict(x_test)
disp3 = confusion_matrix(polyOVOPred, y_test)
print("poly One vs One")
print(disp3)
print("f1 score = ", f1_score(y_test,polyOVOPred,  average = "weighted"), "Accuracy =", np.mean(polyOVOPred == y_test))

polyOVRPred = polyOVR.predict(x_test)
disp4 = confusion_matrix(polyOVRPred, y_test)
print("poly One vs Rest")
print(disp4)
print("f1 score = ", f1_score( y_test,polyOVRPred, average = "weighted"), "Accuracy =", np.mean(polyOVRPred == y_test))

rbfOVOPred = rbfOVO.predict(x_test)
disp5 = confusion_matrix(rbfOVOPred, y_test)
print("rbf One vs One")
print(disp5)
print("f1 score = ", f1_score(y_test,rbfOVOPred,  average = "weighted"), "Accuracy =", np.mean(rbfOVOPred == y_test))

rbfOVRPred = rbfOVR.predict(x_test)
disp6 = confusion_matrix(rbfOVRPred, y_test)
print("rbf One vs Rest")
print(disp6)
print("f1 score = ", f1_score( y_test, rbfOVRPred, average = "weighted"), "Accuracy =", np.mean(rbfOVRPred == y_test))

In [ ]:
w = linearOVR.coef_[0]
a = -w[1]/w[2]
xx = np.linspace(1.5, 4.5)
yy = a * xx - (linearOVR.intercept_[0]) / w[2]
margin = 1 / np.sqrt(np.sum(linearOVR.coef_ **2))
yy_down = yy -np.sqrt(1 + a **2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin
print(linearOVR.coef_[0])

In [ ]:
plt.figure(1, figsize = (7,5))
plt.clf()
plt.plot(xx , yy , "k-")
plt.plot(xx , yy_down, "k-")
plt.plot(xx, yy_up, "k-")
plt.scatter(linearOVR.support_vectors_[:, 1], linearOVR.support_vectors_[:,2], s= 80, facecolors = "none", zorder = 10, edgecolors = "k" )
plt.scatter(x_train[:,1], x_train[:, 2], c = y_train, zorder = 10, cmap = plt.cm.Paired, edgecolors = "k")
plt.xlabel(iris.feature_names[1])
plt.ylabel(iris.feature_names[2])
plt.show()

In [ ]:
w = linearOVR.coef_[0]
a = -w[2]/w[3]
xx = np.linspace(0, 8)
yy = a * xx - (linearOVR.intercept_[0]) / w[3]
margin = 1 / np.sqrt(np.sum(linearOVR.coef_ **2))
yy_down = yy -np.sqrt(1 + a **2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin
print(linearOVR.coef_[0])

In [ ]:
plt.figure(1, figsize = (7,5))
plt.clf()
plt.plot(xx , yy , "k-")
plt.plot(xx , yy_down, "k-")
plt.plot(xx, yy_up, "k-")
plt.scatter(linearOVR.support_vectors_[:, 2], linearOVR.support_vectors_[:,3], s= 80, facecolors = "none", zorder = 10, edgecolors = "k" )
plt.scatter(x_train[:,2], x_train[:, 3], c = y_train, zorder = 10, cmap = plt.cm.Paired, edgecolors = "k")
plt.xlabel(iris.feature_names[2])
plt.ylabel(iris.feature_names[3])
plt.show()